In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd
import networkx as nx

from linecache import getline
from gurobipy import Model, GRB, quicksum
from collections import defaultdict

In [2]:
class FileParser:
#     def __init__(self):
        
    def parseCoords(line):
        x = line.split()
        return [x[0], x[1], x[2]]

    def parseDemand(line):
        x = line.split()
        return [int(x[0]), int(x[1])]

    def parseRoutes(line):
        x = line.split()
        parsedRoutes = x[2:]
        parsedRoutes.insert(0, "1")

        return parsedRoutes

    def readInstance(file):
        COORD_FLAG = False
        DEMAND_FLAG = False

        xc, yc = [], []
        coords, q = {}, {}

        fh = open(file, 'r')
        for i, line in enumerate(fh):
            if "NODE_COORD_SECTION" in line:
                COORD_FLAG = True
            elif "DEMAND_SECTION" in line:
                COORD_FLAG = False
                DEMAND_FLAG = True
            elif "DEPOT_SECTION" in line:
                DEMAND_FLAG = False
            elif COORD_FLAG:
                coord = parseCoords(line)
                xc.append(coord[1])
                yc.append(coord[2])
                coords[coord[0]] = [coord[1], coord[2]]
            elif DEMAND_FLAG:
                demand = parseDemand(line)
                q[demand[0]] = demand[1]
        fh.close()

        return xc, yc, coords, q


    def readSolution(file):
        fh = open(file, 'r')  # A-n32-k5-solution.txt
        routes = {}
        for i, line in enumerate(fh):
            if "Route #" in line:
                routes[i] = parseRoutes(line)
                routes[i] = [x for x in routes[i] ]
                routes[i].append("1")
        return routes

## Reading text files containing instances

In [3]:
fileParser = FileParser()
xc, yc, coords, q = fileParser.readInstance("../Instances/Instances/A-n32-k5.txt")

TypeError: readInstance() takes 1 positional argument but 2 were given

In [ ]:
edges_set = set()

def compareCoordID(u, v):
    return u == v

def checkEdgeExists(tempSet, str1, str2):
    return str1 in tempSet or str2 in tempSet

def stringEdgeBuilder(u, v):
    return "(" + u + ", " + v + ")"

def createEdgeStrings(u, v):
    return stringEdgeBuilder(u, v), stringEdgeBuilder(v, u)

def calculateWeight(coord_u, coord_v):
    return np.hypot(int(coord_u[0]) - int(coord_v[0]), int(coord_u[1]) - int(coord_v[1]))

def getEdgesInOptimalRoute(routes):
    edges_in_optimal_route = set()
    
    for route in routes:
        curr = routes[route]

        for index in range(0, len(curr) - 1):
            coordStr1, coordStr2 = createEdgeStrings(curr[index], curr[index + 1])

            if not checkEdgeExists(edges_in_optimal_route, coordStr1, coordStr2):
                edges_in_optimal_route.add(coordStr1)
    return edges_in_optimal_route

In [ ]:
routes = fileParser.readSolution("../Instances/Solutions/test.txt")
    
edges_in_optimal_route = getEdgesInOptimalRoute(routes)

In [ ]:
def initIncidentMatrix(matrix_size):
    return np.zeros((matrix_size, matrix_size))

In [ ]:
arr = initIncidentMatrix(len(coords))

In [ ]:
def createEdgesDict(coords):
    edges = defaultdict(list)
    dict_global_edge_rank = {}
    index, row, column = 0, 0, 0

    for u in coords:
        for v in coords:
            coordStr1 = stringEdgeBuilder(u, v)
            coordStr2 = stringEdgeBuilder(v, u)

            coord_u = coords[u]
            coord_v = coords[v]
            edge_weight = calculateWeight(coord_u, coord_v)

            if not compareCoordID(u, v) and not checkEdgeExists(edges_set, coordStr1, coordStr2):
                edges_set.add(coordStr1)

                edges["U_X"].append(coord_u[0])
                edges["U_Y"].append(coord_u[1])
                edges["U_NODE_ID"].append(u)

                edges["V_X"].append(coord_v[0])
                edges["V_ Y"].append(coord_v[1])
                edges["V_NODE_ID"].append(v)


                edges["EDGE_WEIGHT"].append(edge_weight)
                dict_global_edge_rank[index] = edge_weight
                index += 1

                if checkEdgeExists(edges_in_optimal_route, coordStr1, coordStr2):
                    edges["IS_OPTIMAL_EDGE"].append(1)
                else:
                    edges["IS_OPTIMAL_EDGE"].append(0)

            arr[row][column] = edge_weight
            column += 1
        column = 0
        row += 1
        
    return edges, dict_global_edge_rank

In [ ]:
edges, dict_global_edge_rank = createEdgesDict(coords)

In [ ]:
def labelMatrix(coords, arr):
    row_labels = [row for row in coords]
    column_labels = [col for col in coords]
    incidence_matrix = pd.DataFrame(arr, columns=column_labels, index=row_labels)
    
    return incidence_matrix

In [ ]:
incidence_matrix = labelMatrix(coords, arr)

In [ ]:
def dfFactory(edges):
    return pd.DataFrame(edges)

In [ ]:
df = dfFactory(edges)
df

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]

graph = df[['U_NODE_ID', 'V_NODE_ID']].where(df['IS_OPTIMAL_EDGE'] == 1)
graph
 
G = nx.from_pandas_edgelist(graph, 'U_NODE_ID', 'V_NODE_ID')

nx.draw(G, with_labels=True)
plt.show()


In [ ]:
def calculateLocalRank(u, v):
    row = incidence_matrix[u]
    sorted_row = row.sort_values(ascending=True)
    return sorted_row[v]

In [ ]:
class Degree:
    def __init__(self, G):
        self.degrees = G.degree
        self.u_node_degree = []
        self.v_node_degree = []
        self.node_average_degree = []
        
    def addDegrees(self, node_u, node_v):
        self.addUDegree(node_u)
        self.addVDegree(node_v)
        self.addAverageDegree(node_u, node_v)
        
    def addUDegree(self, nodeID):
        self.u_node_degree.append(self.degrees[nodeID])
        
    def addVDegree(self, nodeID):
        self.v_node_degree.append(self.degrees[nodeID])
        
    def addAverageDegree(self, node_u, node_v):
        self.node_average_degree.append((self.degrees[node_u] + self.degrees[node_v]) / 2)
        
    def getNodeDegree(self, nodeID):
        return self.degrees[nodeID]
    
    def getAverageNodeDegree(self, node_u, node_v):
        return (self.degrees[node_u] + self.degrees[node_v]) / 2
    
    def addToDF(self, df):
#         u_node_dict = {'U_NODE_DEGREE': self.u_node_degree}
#         df = df.append(pd.DataFrame(u_node_dict))
        df = df.append(pd.DataFrame(data=u_node_degree, columns=['U_NODE_DEGREE']))
        
        print(df)
        
        v_node_dict = {'V_NODE_DEGREE': self.v_node_degree}
        df = df.append(pd.DataFrame(v_node_dict))
        
        print(df)
        
        node_average_dict = {'AVERAGE_NODE_DEGREE': self.node_average_degree}
        df = df.append(pd.DataFrame(node_average_dict))
    
        print(df)
        
        return df

In [ ]:
class EigenvectorCentrality:
    def __init__(self):
        self.u_node_eigenvector_centrality = []
        self.v_node_eigenvector_centrality = []
        self.average_eigenvector_centrality = []
        
    def addUEigenvectorCentrality(eigenvectorCentrality):
        self.u_node_clustering.append(eigenvectorCentrality)
        
    def addVEigenvectorCentrality(eigenvectorCentrality):
        self.v_node_clustering.append(eigenvectorCentrality)
        
    def addAverageEigenvectorCentrality(eigenvectorCentrality):
        self.average_node_clustering.append(eigenvectorCentrality)
        
    def getEigenvectorCentrality(G):
        return nx.eigenvector_centrality(G)
    
    def addToDF(df):
        df['U_NODE_EIGENVECTOR_CENTRALITY'] = u_node_eigenvector_centrality
        df['V_NODE_EIGENVECTOR_CENTRALITY'] = v_node_eigenvector_centrality
        df['AVERAGE_NODE_EIGENVECTOR_CENTRALITY'] = average_eigenvector_centrality
    
        return df

In [ ]:
class Clustering:
    def __init__(self, G):
        self.clustering = nx.clustering(G)
        self.u_node_clustering = []
        self.v_node_clustering = []
        self.average_node_clustering = []
    
    def addClusterings(self, node_u, node_v):
        self.addUClustering(node_u)
        self.addVClustering(node_v)
        self.addAverageClustering(node_u, node_v)        
        
    def addUClustering(self, nodeID):
        self.u_node_clustering.append(self.clustering[nodeID])
        
    def addVClustering(self, nodeID):
        self.v_node_clustering.append(self.clustering[nodeID])
        
    def addAverageClustering(self, node_u, node_v):
        self.average_node_clustering.append((self.clustering[node_u] + self.clustering[node_v]) / 2)
        
    def getClustering():
        return self.clustering
    
    def addToDF(self, df):
        u_node_dict = {'U_NODE_CLUSTERING': self.u_node_clustering}
        df = df.append(pd.DataFrame(u_node_dict))
        
        v_node_dict = {'V_NODE_CLUSTERING': self.v_node_clustering}
        df = df.append(pd.DataFrame(v_node_dict))
        
        node_average_dict = {'AVERAGE_NODE_CLUSTERING': self.average_node_clustering}
        df = df.append(pd.DataFrame(node_average_dict))
    
        return df

In [ ]:
class DegreeCentrality:
    def __init__(self):
        self.u_node_degree_centrality = []
        self.v_node_degree_centrality = []
        self.average_degree_centrality = []
        
    def addUDegreeCentrality(degreeCentrality):
        self.u_node_degree_centrality.append(degreeCentrality)
        
    def addVDegreeCentrality(degreeCentrality):
        self.v_node_degree_centrality.append(degreeCentrality)
        
    def addAverageDegreeCentrality(degreeCentrality):
        self.average_degree_centrality.append(degreeCentrality)
        
    def getClustering(G):
        return nx.clustering(G)
    
    def addToDF(df):
        df['U_NODE_DEGREE_CENTRALITY'] = u_node_degree_centrality
        df['V_NODE_DEGREE_CENTRALITY'] = u_node_degree_centrality
        df['AVERAGE_NODE_DEGREE_CENTRALITY'] = average_degree_centrality
    
        return df 

In [ ]:
# Feature flags

degreeFlag = True
# eigenvector_centrality = True
clusteringFlag = True
# degree_centrality = True
global_rank = True
local_rank = True

In [ ]:
u_node_degree = []
v_node_degree = []
node_average_degree = []

global_edge_rank = []

u_node_local_edge_rank = []
v_node_local_edge_rank = []

# u_node_eigenvector_centrality = []
# v_node_eigenvector_centrality = []
# average_eigenvector_centrality = []

u_node_clustering = []
v_node_clustering = []
average_node_clustering = []

# u_node_degree_centrality = []
# v_node_degree_centrality = []
# average_degree_centrality = []

# if eigenvector_centrality:
#     eigenvector_centrality = nx.eigenvector_centrality(G)

# if clustering:
#     clustering = nx.clustering(G)

# if degree_centrality:
#     degree_centrality = nx.degree_centrality(G)

degree = Degree(G)
clustering = Clustering(G)
    
for index, row in df.iterrows():
    if degreeFlag:
        node_u, node_v = row["U_NODE_ID"], row["V_NODE_ID"]
        degree.addDegrees(node_u, node_v)
    
#     if eigenvector_centrality:
#         node_u = eigenvector_centrality[row["U_NODE_ID"]]
#         u_node_eigenvector_centrality.append(node_u)
        
#         node_v = eigenvector_centrality[row["V_NODE_ID"]]
#         v_node_eigenvector_centrality.append(node_v)
        
#         average_eigenvector_centrality.append((node_u + node_v) / 2)

    if clusteringFlag:
        node_u, node_v = row["U_NODE_ID"], row["V_NODE_ID"]
        clustering.addClusterings(node_u, node_v)
    
#     if degree_centrality:
#         node_u = degree_centrality[row["U_NODE_ID"]]
#         u_node_degree_centrality.append(node_u)
        
#         node_v = degree_centrality[row["V_NODE_ID"]]
#         v_node_degree_centrality.append(node_v)
        
#         average_degree_centrality.append((node_u + node_v) / 2)
        
    if global_rank:
        global_edge_rank.append(dict_global_edge_rank[index])
    
    if local_rank:
        u_node_local_edge_rank.append(calculateLocalRank(row["U_NODE_ID"], row["V_NODE_ID"]))
        v_node_local_edge_rank.append(calculateLocalRank(row["V_NODE_ID"], row["U_NODE_ID"]))

if degreeFlag:
    df = degree.addToDF(df)

# if eigenvector_centrality:
#     df['U_NODE_EIGENVECTOR_CENTRALITY'] = u_node_eigenvector_centrality
#     df['V_NODE_EIGENVECTOR_CENTRALITY'] = v_node_eigenvector_centrality
#     df['AVERAGE_NODE_EIGENVECTOR_CENTRALITY'] = average_eigenvector_centrality

if clusteringFlag:
    df = clustering.addToDF(df)
#     df['U_NODE_CLUSTERING'] = u_node_clustering
#     df['V_NODE_CLUSTERING'] = v_node_clustering
#     df['AVERAGE_NODE_CLUSTERING'] = average_node_clustering

# if degree_centrality:
#     df['U_NODE_DEGREE_CENTRALITY'] = u_node_degree_centrality
#     df['V_NODE_DEGREE_CENTRALITY'] = u_node_degree_centrality
#     df['AVERAGE_NODE_DEGREE_CENTRALITY'] = average_degree_centrality
    
# if global_rank:
#     df['GLOBAL_EDGE_RANK'] = global_edge_rank

# if local_rank:
#     df['U_NODE_LOCAL_EDGE_RANK'] = u_node_local_edge_rank
#     df['V_NODE_LOCAL_EDGE_RANK'] = v_node_local_edge_rank

In [ ]:
df

In [ ]:
# df['GLOBAL_EDGE_RANK'] == '1'